для решения необходимо задать  
широту, долготу и альтитуду  
места установки ветрогенератора

In [99]:
# uncomment for manual entry

# lat_test = float(input('enter lattitude ')) # in format xx.xxxx (deg)
# lon_test = float(input('enter longitude ')) # in format xx.xxxx (deg)
# alt_test = int(input('enter altitude ')) # in format xx (m)


в учебных целях используются координаты м.Таран в Калининградской области,  
не затенено поверхностью порядка 300 градусов,  
место открыто для всех преобладающих ветров в данном регионе  

In [183]:
lat_test = 54.9593
lon_test = 19.9893
alt_test = 30

библиотека folium, для отображения карты

In [184]:
import folium
poi = folium.Map(location=[lat_test, lon_test], zoom_start=7)
folium.Marker([lat_test, lon_test]).add_to(poi)
poi

In [ ]:
# lat_train =
# lon_train = 
# alt_train =

In [168]:
# lat_test, lon_test, alt_test

по географическим данным можем получить ближайшую метеостанцию,  
которая выдает почасовые данные  
устанавливаем и импортируем библиотеку meteostat  
также необходима библиотека datetime для работы со временем  

библиотека metestat еще удобна тем, что выводит данные сразу в DataFrame,  
нет необходимости обрабатывать запросы и ответы по API,
хотя и API так же имеется

In [78]:
# pip install meteostat
import meteostat
from meteostat import Stations
from datetime import datetime

### meteostat library is provided by:  
https://dev.meteostat.net/  

### Quota  
Free access to the Meteostat API is  
### limited to 2,000 requests per day and no more than two requests per second.  
Once this limit is reached, all of your requests will result in HTTP 429, too many requests, until your quota is cleared.  

достаточно для целей проекта


In [79]:
# help(meteostat)

список станций, передающих данные ежечасно на указанную дату

In [80]:
st = Stations()
st = st.inventory('hourly', datetime(2021,1,1))
# st.fetch()['region'].value_counts()
st_1 = st.fetch()
st.fetch().shape

(10325, 13)

на 1/01/2021 работало 10325 метеостанций по всему миру

описание данных:  
id - идентификатор WMO (World Meteoroligical Organization, Всемирная метеорологическая организация)  
name - английское название метеостанции  
country - страна, согласно ISO 3166-1 alpha-2 country code of the weather station  
region - регион - согласно ISO  3166-2 state or region code of the weather station  
wmo - см. id  
icao - International Civil Aviation Organization (Международная Организация Гражданской Авиации) - если станция установлена в аэропорту        
latitude - широта метеостанции  
longitude - долготоа метеостанции  
elevation - высота метеостанции над уровнем моря  
timezone - временная зона  
hourly_start - дата начала передачи метео данных в почасовом режиме  
hourly_end - дата окончания передачи метео данных в почасовом режиме  
daily_start - датат начала передачи данных в суточном режима  
daily_end - дата окончания передачи данных в суточном режиме  

In [89]:
st.fetch().head()

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,distance
id,,,,,,,,,,,,,,
01001,Jan Mayen,NO,<NA>,01001,ENJA,70.9333,-8.6667,10.0,Europe/Oslo,1955-01-01,2021-02-23,1922-01-01,2021-02-10,2.292450e+06
01003,Hornsund,NO,<NA>,01003,<NA>,77.0000,15.5000,10.0,Europe/Oslo,1985-06-01,2021-02-23,NaT,NaT,2.459223e+06
01008,Svalbard Lufthavn,NO,<NA>,01008,ENSB,78.2500,15.4667,2.0,Europe/Oslo,1975-09-29,2021-02-23,1975-08-01,2021-02-10,2.597494e+06
01010,Andoya,NO,NO,01010,ENAN,69.3000,16.1500,1.0,Europe/Oslo,1973-01-01,2021-02-23,2015-09-07,2018-12-28,1.607053e+06
01015,Hekkingen Lighthouse,NO,TR,01015,<NA>,69.6000,17.8333,14.0,Europe/Oslo,1980-03-14,2021-02-23,1979-11-01,2021-02-10,1.631786e+06


получим список метеостанций, рядом с интересующим местом,  
в радиусе 100 км  
при избыточном количестве метеосатнций, можно ограничить вывод,  
указаав требуемое количество в методе fetch(n)   
n - требуемое количество метеостанций,  
метод выводит n самых ближайших, отсортировано по расстоянию

In [98]:
dist_test = 100000 # in meters
st.nearby(lat_test, lon_test, dist_test).fetch()

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,distance
id,,,,,,,,,,,,,,
26702,Kaliningrad,RU,KGD,26702,UMKK,54.7167,20.5500,27.0,Europe/Kaliningrad,1955-01-01,2021-02-23,1931-01-01,2021-02-07,44909.570165
26603,Nida,LT,KP,26603,<NA>,55.3167,21.0167,2.0,Europe/Vilnius,2018-01-28,2021-02-23,1959-01-01,2014-09-08,76443.047043
12135,Hel,PL,PM,12135,<NA>,54.6000,18.8167,1.0,Europe/Warsaw,1955-07-01,2021-02-23,NaT,NaT,85151.691663
12160,Elblag,PL,WM,12160,EPEL,54.1667,19.4333,40.0,Europe/Warsaw,1953-01-02,2021-02-23,1936-01-03,2021-02-07,95144.078495
12155,Gdansk-Swibno,PL,PM,12155,<NA>,54.3333,18.9333,7.0,Europe/Warsaw,1982-09-01,2021-02-23,NaT,NaT,97270.314253


для моделирования будут использоваться данные метеостанции,  
расположенной в аэропорту Храброво (icao = UMKK),  
как наиболее близкорасположенная к объекту исследования  

по остальным станциям будет проводится учеба и оценка модели

In [103]:
st.nearby(lat_test, lon_test, dist_test).fetch(1)

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,distance
id,,,,,,,,,,,,,,
26702,Kaliningrad,RU,KGD,26702,UMKK,54.7167,20.55,27.0,Europe/Kaliningrad,1955-01-01,2021-02-23,1931-01-01,2021-02-07,44909.570165


получим время начала записи ежечасных данных,  
как максимум по hourly_start

In [173]:
for_date = st.nearby(lat_test, lon_test, dist_test).fetch()
start = for_date['hourly_start'].max()

now = datetime.now().strftime('%Y,%m,%d %H:%M')

end = datetime.strptime(now,'%Y,%m,%d %H:%M' )
# start.date(), now, end

In [174]:
from meteostat import Hourly

data = Hourly('26702', start, end)
data = data.fetch()
data

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2018-01-28 00:00:00,3.0,1.7,91.0,NaN,NaN,180.0,3.6,NaN,1025.7,NaN,4.0
2018-01-28 01:00:00,3.0,2.0,93.0,NaN,NaN,180.0,18.0,NaN,1023.0,NaN,NaN
2018-01-28 02:00:00,3.0,2.0,93.0,NaN,NaN,190.0,18.0,NaN,1022.0,NaN,NaN
2018-01-28 03:00:00,3.0,1.0,87.0,NaN,NaN,190.0,7.2,NaN,1021.1,NaN,NaN
2018-01-28 04:00:00,3.0,2.0,93.0,NaN,NaN,170.0,21.6,NaN,1018.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-18 13:00:00,-6.0,-11.9,63.0,0.0,NaN,126.0,9.3,20.4,1026.4,NaN,1.0
2021-02-18 14:00:00,-6.1,-11.8,64.0,0.0,NaN,127.0,9.3,20.4,1026.2,NaN,1.0
2021-02-18 15:00:00,-6.3,-11.8,65.0,0.0,NaN,129.0,9.3,18.5,1026.0,NaN,1.0


описание данных:  
temp - температура воздуха в градусах Цельсия  
dwpt - точка росы в градусах Цельсия  
rhum - относительная влажность в %  
prcp - осадки в час, в мм   
snow - глубина снежного покрова в мм  
wdir - направление ветра в градусах  
wspd - скорость ветра в км/ч    
wpgt - пиковая скорость ветра в порывах в км/ч   
press - давление в hPa  
tsun - солнечно в течение часа в минутах  
coco - код условий погоды, см.ссылку ниже  

https://dev.meteostat.net/docs/formats.html#weather-condition-codes
